<a href="https://colab.research.google.com/github/SadhnaBijrothiya/DL/blob/main/DataCode_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import xlrd
import pandas as pd
import os
import matplotlib.pyplot as plt
import io
import math
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from scipy.signal import savgol_filter
from scipy import signal
from scipy.spatial import distance
from sklearn.model_selection import train_test_split

# Create the LSTM model

In [ ]:
from tensorflow import keras, convert_to_tensor, float32, make_ndarray
from tensorflow.compat.v1 import Session
from tensorflow.keras import layers, backend
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalAveragePooling1D, Dropout
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, MaxPool1D, TimeDistributed, RepeatVector
from keras.layers.merge import concatenate
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WBDS01walkT01ang.txt to WBDS01walkT01ang (4).txt


In [ ]:
def plotData(person, speed):
  ini_name = "/content/" 
  name ='WBDS' + ("" if person > 9 else "0") + str(person) + 'walkT' + "0" + str(speed)
  data = pd.read_csv(ini_name + name + 'ang.txt', delimiter = "\t")
  plt.plot(data['LKneeAngleZ'], label='Left Knee')
  plt.plot(data['LAnkleAngleZ'], label='Left Ankle')
  plt.plot(data['LHipAngleZ'], label='Left Hip')
  plt.ylabel("Angle")
  plt.xlabel("Time Steps")
  plt.legend()
  plt.show()
  print(data)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WBDSinfo.xlsx to WBDSinfo (1).xlsx


In [ ]:
personalData = pd.read_excel(r"/content/WBDSinfo.xlsx", engine='openpyxl')

In [ ]:
personalData

,Subject,FileName,AgeGroup,Age,Height,Mass,Gender,Dominance,LegLength,Static1,Static2,GaitSpeed(m/s),TreadHands,FP_RightFoot,FP_LeftFoot,Notes,BorgScale,Unnamed: 17
0,1,WBDS01static1.c3d,Young,25,172.5,74.30,M,R,0.89,Yes,No,--,--,--,--,--,10.0,NaN
1,1,WBDS01walkT01.c3d,Young,25,172.5,74.30,M,R,0.89,Yes,No,0.49,No,--,--,--,10.0,NaN
2,1,WBDS01walkT02.c3d,Young,25,172.5,74.30,M,R,0.89,Yes,No,0.67,No,--,--,--,10.0,NaN
3,1,WBDS01walkT03.c3d,Young,25,172.5,74.30,M,R,0.89,Yes,No,0.85,No,--,--,--,10.0,NaN
4,1,WBDS01walkT04.c3d,Young,25,172.5,74.30,M,R,0.89,Yes,No,1.03,No,--,--,--,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6910,42,WBDS42walkT06knt.txt,Older,63,161.2,59.85,F,R,0.82,--,--,--,--,--,--,--,9.0,NaN
6911,42,WBDS42walkT07ang.txt,Older,63,161.2,59.85,F,R,0.82,--,--,--,--,--,--,--,9.0,NaN
6912,42,WBDS42walkT07knt.txt,Older,63,161.2,59.85,F,R,0.82,--,--,--,--,--,--,--,9.0,NaN
6913,42,WBDS42walkT08ang.txt,Older,63,161.2,59.85,F,R,0.82,--,--,--,--,--,--,--,9.0,NaN


In [ ]:
filter = personalData["FileName"]=="WBDS01walkT01.c3d"

# filtering data 

In [ ]:
dt = personalData.loc[filter]

# display

In [ ]:
dt.iloc[0]['Age']

25

In [ ]:
def todot(a,dt):
  x=len(a)
  b=numpy.zeros(a.shape)
  b[0]=a[0]/dt
  for i in range(1,x):
    b[i]=(a[i]-a[i-1])/dt
  return b

In [ ]:
def plotDot(jointAngle, jointAngleDot, time):
  fig = plt.figure(figsize=(10,10))
  ax = fig.gca(projection='3d')
  z = numpy.linspace(0, time/100, time - 1)
  ax.plot(jointAngle, jointAngleDot, z)
  ax.set_xlabel("joint angle")
  ax.set_ylabel("angular velocity")
  plt.show()

In [ ]:
def custom_loss(y_true, y_pred):
                                       # print(y_true[0][0], y_pred[0][0])
  initial = []
  rmse = 0
  n = len(y_true)
  ty = y_true[:, 100]
  py = y_pred[:, 0]
  return (backend.mean(backend.square(y_pred - y_true)) / 101)

In [ ]:
def build_lstm(time_steps, features):
  model = Sequential() #sequential model
  model.add(LSTM(100, input_shape=(time_steps, features), return_sequences=False))
  # model.add(RepeatVector(3))
  model.add(Dense(50, activation='relu'))
  # model.add(Dense(50, activation='relu'))
  model.add(Dense(time_steps, activation='linear'))
  model.compile(optimizer='adam', loss=custom_loss,metrics=['accuracy'])
  model.summary()
  return model

#CNN model

In [ ]:
def build_cnn(time_steps, features):
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(time_steps, features)))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(50, activation='relu'))
  model.add(Dense(time_steps, activation='linear'))
  model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])
  print(model.summary())
  return model

#CNN_GRU model implementation

In [ ]:
def build_cnn_gru(time_steps, features):
  input = keras.Input(shape=(time_steps, features))
  convModel = Conv1D(filters=64, kernel_size=2, activation='relu')(input)
  # convModel = Dropout(0.2)(convModel)
  convModel = MaxPooling1D(pool_size=2)(convModel)
  convModel = Conv1D(filters=64, kernel_size=2, activation='relu')(convModel)
  convModel = MaxPooling1D(pool_size=2)(convModel)
  # convModel = Dropout(0.2)(convModel)
  convModel = Flatten()(convModel)

  ##second conv model
  convModel2 = Conv1D(filters=64, kernel_size=2, activation='relu')(input)
  # convModel = Dropout(0.2)(convModel)
  convModel2 = MaxPooling1D(pool_size=2)(convModel2)
  convModel2 = Conv1D(filters=64, kernel_size=2, activation='relu')(convModel2)
  convModel2 = MaxPooling1D(pool_size=2)(convModel2)
  convModel2 = Conv1D(filters=64, kernel_size=2, activation='relu')(convModel2)
  convModel2 = MaxPooling1D(pool_size=2)(convModel2)
  # convModel = Dropout(0.2)(convModel)
  convModel2 = Flatten()(convModel2)

  gruModel = GRU(100, return_sequences=False)(input)

  merge = concatenate([convModel, convModel2, gruModel])
  # merge = GlobalAveragePooling1D()(merge)

  hidden1 = Dense(time_steps, activation='relu')(merge)
  output = Dense(time_steps, activation='linear')(hidden1)
  # output = GlobalAveragePooling1D()(hidden1)

  model = Model(inputs=input, outputs=output)
  print(model.summary())
  model.compile(
      optimizer='adam',
      loss=custom_loss,
      metrics=['accuracy']
  )
  return model

In [ ]:
def modelPlot(model, name):
  plot_model(model, to_file=name+'.png', show_shapes=True)

In [ ]:
build_lstm(101, 7)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               43200     
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 101)               5151      
Total params: 53,401
Trainable params: 53,401
Non-trainable params: 0
_________________________________________________________________


In [ ]:
temp = pd.read_csv('/content/WBDS02walkT01ang.txt')

FileNotFoundError: ignored